In [2]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip3 install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip3 install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip3 install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip3 install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip3 install torch-geometric 


/home/parham/anaconda3/lib/python3.8/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/parham/anaconda3/lib/python3.8/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
/home/parham/anaconda3/lib/python3.8/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/parham/anaconda3/lib/python3.8/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Looking in links: https://pytorch-geometric.com/whl/torch-1.9

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
from torch_geometric.utils import to_dense_adj

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

OSError: libcusparse.so.11: cannot open shared object file: No such file or directory

In [2]:
import torch_geometric.datasets.planetoid
dataset = torch_geometric.datasets.amazon.Amazon('.','Computers')


ModuleNotFoundError: ignored

In [4]:
d = DataLoader(dataset,1,True)

In [5]:
class GNNStack(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, task='node'):
        super(GNNStack, self).__init__()
        self.task = task
        self.conv1 = pyg_nn.GCNConv(input_dim, hidden_dim)
        self.conv2 = pyg_nn.GCNConv(hidden_dim, output_dim)

        self.dropout = 0.25
        self.num_layers = 3



    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        if data.num_node_features == 0:
          x = torch.ones(data.num_nodes, 1)

        x = self.conv(x,edge_index)
        x = F.relu(x)
        

        return F.sigmoid(x@torch.transpose(x, 0, 1))



In [10]:
device = torch.device('cuda:0')
print(device)
model = GNNStack(dataset.data.x.shape[1],6,0).to(device)
opt = optim.Adam(model.parameters(), lr=0.01)
loss = nn.BCELoss()


cuda:0


RuntimeError: ignored

In [ ]:
data = dataset.data.to(device)
for ep in range(50):
    pred = model(data)
    l = loss(pred.view(1,-1),to_dense_adj(dataset.data.edge_index).view(1,-1))
    opt.zero_grad()
    l.backward()
    opt.step()
    print(ep,l)

In [125]:
import sklearn.metrics as metrics
pred = model(dataset.data)



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [146]:
to_dense_adj(dataset.data.edge_index).view(1,-1).shape

torch.Size([1, 189117504])